In [1]:
import pandas as pd
from sqlalchemy import create_engine
from secret import user_pass 
import psycopg2

# Extract CSV into DataFrames

In [2]:
og_file= "./Resources/Oil and Gas 1932-2014.csv"
og=pd.read_csv(og_file)
og.head()

,cty_name,iso3numeric,id,year,eiacty,oil_prod32_14,oil_price_2000,oil_price_nom,oil_value_nom,oil_value_2000,...,net_gas_exports_mboe,net_gas_exports_value,net_gas_exports_valuePOP,net_oil_gas_exports_valuePOP,population,pop_maddison,sovereign,mult_nom_2000,mult_nom_2014,mult_2000_2014
0,Afghanistan,4,AFG,1932,Afghanistan,NaN,10.860860,0.87,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,12.599874,17.321973,1.374774
1,Afghanistan,4,AFG,1933,Afghanistan,NaN,8.815239,0.67,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,13.280064,18.257080,1.374774
2,Afghanistan,4,AFG,1934,Afghanistan,NaN,12.715790,1.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,12.853008,17.669975,1.374774
3,Afghanistan,4,AFG,1935,Afghanistan,NaN,12.039690,0.97,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,12.537882,17.236748,1.374774
4,Afghanistan,4,AFG,1936,Afghanistan,NaN,13.383210,1.09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,12.420786,17.075768,1.374774


In [3]:
inf_file="./Resources/inflation_consumer_prices.xlsx"
inflation=pd.read_excel(inf_file)
inflation.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,8.957732,-2.136372,2.077739,4.374596,0.571756,-2.372263,0.420998,0.476485,-0.893020,NaN
1,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,30.554941,-8.283078,0.892537,10.201660,7.218258,7.654317,4.604334,-1.533847,2.169452,NaN
2,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,12.473713,13.731451,14.470541,13.467015,10.293494,8.776091,7.279562,10.279327,34.736159,NaN
3,Albania,ALB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.359242,2.280502,3.552267,3.450347,2.031596,1.937618,1.631778,1.910017,1.282799,NaN
4,Andorra,AND,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Transform DataFrames


In [4]:
# Create a filtered dataframe from specific columns
og_cols = ["cty_name","id","year","oil_price_2000","oil_value_2000","gas_price_2000","population"]
og_transformed= og[og_cols].copy()

# Extract 2000 
og_transformed=og_transformed[og_transformed.year==2000]


# Rename the column headers
og_transformed = og_transformed.rename(columns={"cty_name": "city_name","oil_price_2000":"oil_price",
                                                "oil_value_2000":"oil_value","gas_price_2000": "gas_price","year":"years"})
og_transformed.head()

,city_name,id,years,oil_price,oil_value,gas_price,population
68,Afghanistan,AFG,2000,28.50001,0.000000e+00,3.703766,20595360.0
151,Albania,ALB,2000,28.50001,6.621231e+07,3.703766,3060256.0
234,Algeria,DZA,2000,28.50001,1.395263e+10,3.703766,30458956.0
317,Angola,AGO,2000,28.50001,7.602928e+09,3.703766,13833075.0
400,Argentina,ARG,2000,28.50001,8.396640e+09,3.703766,36995376.0


In [23]:
# Group By Country "Id"
og_transformed=og_transformed.groupby(['id']).mean().reset_index()
og_transformed.head()


,id,years,oil_price,oil_value,gas_price,population
0,AFG,2000,28.50001,0.000000e+00,3.703766,20595360.0
1,AGO,2000,28.50001,7.602928e+09,3.703766,13833075.0
2,ALB,2000,28.50001,6.621231e+07,3.703766,3060256.0
3,ARE,2000,28.50001,2.443798e+10,3.703766,3243812.0
4,ARG,2000,28.50001,8.396640e+09,3.703766,36995376.0


In [24]:
# Export clean data to csv format
og_transformed.to_csv (r'oil_gas_price.csv', index = None, header=True)

In [57]:
# Create a filtered dataframe from specific columns
inflat_cols = ["Country Name","Country Code",2000]
inflation_transformed=inflation[inflat_cols].copy()


# Rename the column headers
inflation_transformed = inflation_transformed.rename(columns={"Country Code":"id", 2000:"inflation","Country Name":"country"})
inflation_transformed.head()


,country,id,inflation
0,Aruba,ABW,4.044021
1,Afghanistan,AFG,NaN
2,Angola,AGO,324.996872
3,Albania,ALB,0.050018
4,Andorra,AND,NaN


In [58]:
# Remove rows have NAs
inflation_transformed=inflation_transformed.dropna()


inflation_transformed.head()

,country,id,inflation
0,Aruba,ABW,4.044021
2,Angola,AGO,324.996872
3,Albania,ALB,0.050018
5,Arab World,ARB,1.853790
7,Argentina,ARG,-0.935939


In [60]:
# Export clean data to csv format
inflation_transformed.to_csv (r'inflation.csv', index = None, header=True)

# Create database connection

In [64]:
engine = create_engine(f"postgresql://{user_pass}@localhost:5432/oil_gas_prices_inflation_db")
engine.table_names()

['oil_gas', 'inflation']

# Use pandas to load csv converted DataFrame into database

In [65]:
og_transformed.to_sql(name='oil_gas', con=engine, if_exists='append', index=False)


In [67]:
inflation_transformed.to_sql(name='inflation', con=engine, if_exists='append', index=False)

# Confirm data has been added by querying 

In [68]:
pd.read_sql_query('select * from oil_gas', con=engine).head()

,id,years,oil_price,oil_value,gas_price,population
0,AFG,2000,28.5,0.000000e+00,3.70377,20595360.0
1,AGO,2000,28.5,7.602930e+09,3.70377,13833075.0
2,ALB,2000,28.5,6.621230e+07,3.70377,3060256.0
3,ARE,2000,28.5,2.443800e+10,3.70377,3243812.0
4,ARG,2000,28.5,8.396640e+09,3.70377,36995376.0


In [69]:
pd.read_sql_query('select * from inflation', con=engine).head()

,id,country,inflation
0,ABW,Aruba,4.044020
1,AGO,Angola,324.997000
2,ALB,Albania,0.050018
3,ARB,Arab World,1.853790
4,ARG,Argentina,-0.935939


In [74]:
pd.read_sql_query('select * from oil_gas join inflation on oil_gas.id=inflation.id', con=engine).head()


,id,years,oil_price,oil_value,gas_price,population,id,country,inflation
0,AGO,2000,28.5,7.602930e+09,3.70377,13833075,AGO,Angola,324.997000
1,ALB,2000,28.5,6.621230e+07,3.70377,3060256,ALB,Albania,0.050018
2,ARG,2000,28.5,8.396640e+09,3.70377,36995376,ARG,Argentina,-0.935939
3,ARM,2000,28.5,0.000000e+00,3.70377,3042556,ARM,Armenia,-0.790884
4,AUS,2000,28.5,7.394060e+09,3.70377,19133087,AUS,Australia,4.475180
